In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification 
import pandas as pd
import librosa
import numpy as np
from tqdm import tqdm

import sys
sys.path.append("..")
from src.models import EModel, AASIST, Wav2Vec2Facebook
from src.datamodules import AASIST2DataModule, AASISTDataset

In [3]:
# model_name_or_path = "facebook/wav2vec2-base"
# feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)
# sampling_rate = feature_extractor.sampling_rate

path = "/home/work/joono/joono/joono/DV_DV.Deep/mq9cmikn/checkpoints/best-checkpoint_aug_oneshot.ckpt"
# model = Wav2Vec2Facebook.load_from_checkpoint(path, args={})
model = AASIST.load_from_checkpoint(path, args={})

In [4]:
# Collate 함수 정의
def collate_fn(batch):
    # signals = zip(*batch)
    signals = batch
    max_length = max([signal.size(0) for signal in signals])
    padded_signals = torch.zeros(len(signals), max_length)
    for i, signal in enumerate(signals):
        padded_signals[i, :signal.size(0)] = signal
    # labels = torch.tensor(labels)
    return padded_signals

In [5]:
test_df = pd.read_csv('../dataset/test.csv', index_col=None)
test_df['path'] = '../dataset/' + test_df['path'].str[1:]
# test_dataset = TestDataset(test_df)
test_dataset = AASISTDataset(test_df, model, train_mode=False)
# test_loader = DataLoader(test_dataset, shuffle=False, num_workers=24, batch_size=32)
test_loader = DataLoader(test_dataset, shuffle=False, num_workers=24, batch_size=32, collate_fn=collate_fn)

In [6]:
def inference(model, test_loader, device):
    model.to(device)
    model = model.eval()
    predictions = []
    with torch.no_grad():
        # for inputs, labels in tqdm(test_loader):
        for inputs in tqdm(test_loader):
            
            inputs = inputs.to(device)
            # labels = labels.to(device)
            
            probs = model(inputs)

            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
            
    return predictions

In [7]:
preds = inference(model=model, test_loader=test_loader, device='cuda:1')
# preds = model.inference(test_loader=test_loader)

  0%|          | 0/1563 [00:00<?, ?it/s]/home/work/joono/joono/joono/src/AASIST.py:408: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608847532/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(x,
/home/work/joono/anaconda3/envs/torch/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608847532/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 1563/1563 [04:36<00:00,  5.64it/s]


In [8]:
submit = pd.read_csv('/home/work/joono/joono/dataset/sample_submission.csv')

max_thres = 0.999
min_thres = 0.001

for i in tqdm(range(len(preds))):
    if      preds[i][0] > max_thres : submit.iloc[i, 1] = 1
    elif    preds[i][0] < min_thres : submit.iloc[i, 1] = 0 
    else                            : submit.iloc[i, 1] = preds[i][0]
    if      preds[i][1] > max_thres : submit.iloc[i, 2] = 1
    elif    preds[i][1] < min_thres : submit.iloc[i, 2] = 0 
    else                            : submit.iloc[i, 2] = preds[i][1]

submit[1000:1050]

  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_437042/1721487462.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.02107398398220539' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  else                            : submit.iloc[i, 1] = preds[i][0]
/tmp/ipykernel_437042/1721487462.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.937558650970459' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  else                            : submit.iloc[i, 2] = preds[i][1]
100%|██████████| 50000/50000 [00:08<00:00, 6106.76it/s]


,id,fake,real
1000,TEST_01000,0.249968,1.000000
1001,TEST_01001,0.146720,0.997085
1002,TEST_01002,0.635462,1.000000
1003,TEST_01003,0.081568,0.998551
1004,TEST_01004,0.866925,1.000000
1005,TEST_01005,0.029874,0.997929
1006,TEST_01006,0.082814,1.000000
1007,TEST_01007,0.056049,1.000000
1008,TEST_01008,0.011366,1.000000
1009,TEST_01009,0.003258,1.000000


In [ ]:
# submit.to_csv('j0.5.csv', index=False)

In [9]:
submit.to_csv('joono_AASIST_lora_train_test_align_test_submit2.csv', index=False)

In [ ]:
preds